In [1]:
# Qpt

In [4]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
start_all = time.time()

In [7]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [8]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [9]:
# True Object
true_objects = []
true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [10]:
num_data = [100, 1000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=True, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=False, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=True, seed=seed
    ),
    StandardQpt(
        tester_states, tester_povms, on_para_eq_constraint=False, seed=seed
    ),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  1%|          | 1/100 [00:00<00:10,  9.28it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator


  2%|▏         | 2/100 [00:00<00:06, 15.71it/s]

elapsed_time:0.10476798216501872[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]

elapsed_time:0.11430695056915283[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]

elapsed_time:0.7356993635495503[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:42<00:00,  2.35it/s]

elapsed_time:0.7084565838177999[min]



In [51]:
fig = data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    show_analytical_results=True,
    estimator_list=estimator_list,
)
fig.show()

In [52]:
report.export_report(
    "sample_qpt.pdf",
    estimation_results_list,
    case_name_list,
    estimator_list,
    true_object,
    tester_states + tester_povms,
    seed=seed,
    computation_time=sum(elapsed_times)
)

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


100%|██████████| 100/100 [00:00<00:00, 35790.63it/s]


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



​​Generating physicality violation test blocks ...
Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_qpt.pdf)


In [39]:
func = make_graphs_trace_error
arg_names = func.__code__.co_varnames[:func.__code__.co_argcount]

arg_names

('estimation_results', 'num_data_index', 'bin_size')

In [247]:
figs = physicality_violation_check.make_graphs_eigenvalues(estimation_results_list[0], true_object=true_object, bin_size=0.0001)

In [248]:
figs[2]

In [235]:
figs[1]

In [12]:
# Debug

In [12]:
result = estimation_results_list[0][0]

In [19]:
est = result.estimated_qoperation

In [20]:
est.hs

array([[ 1.  ,  0.  ,  0.  ,  0.  ],
       [-0.08,  1.08,  0.12, -0.02],
       [-0.01,  0.11,  1.01, -0.03],
       [ 0.  , -0.22,  0.04,  1.  ]])

In [21]:
choi_matrix = est.to_choi_matrix()
eigenvals, eigenvecs = np.linalg.eig(choi_matrix)

In [22]:
eigenvals

array([ 2.0514533 -2.95797181e-18j,  0.16522093+5.52794968e-17j,
       -0.03228602-9.58242101e-17j, -0.18438821+3.74311530e-17j])

In [37]:
def make_graphs_trace_error(
    estimation_results: List["EstimatedResult"],
    num_data_index: int,
    bin_size: float = 0.0001,
):
    num_data = estimation_results[0].num_data
    estimated_gates = physicality_violation_check._convert_result_to_qoperation(
        estimation_results, num_data_index=num_data_index
    )
    size = estimated_gates[0].dim ** 2
    expected = np.zeros((1, size))
    expected[0][0] = 1
    expected = expected.flatten().tolist()
    figs = []
    for i in range(size):
        values = [gate.hs[0][i] for gate in estimated_gates]
        print(values)

        fig = physicality_violation_check.make_prob_dist_histogram(
            values, bin_size=bin_size, num_data=num_data, annotation_vlines=[expected[i]]
        )
        title = f"N={num_data[num_data_index]}, α={i}"
        fig.update_layout(title=title)
        figs.append(fig)
    return figs

In [66]:
def make_graphs_trace_error_sum(
    estimation_results: List["EstimatedResult"],
    num_data_index: int,
    bin_size: float = 0.0001,
):
    num_data = estimation_results[0].num_data
    estimated_gates = physicality_violation_check._convert_result_to_qoperation(
        estimation_results, num_data_index=num_data_index
    )
    size = estimated_gates[0].dim ** 2
    expected = np.zeros((1, size))
    expected[0][0] = 1
    expected = expected.flatten().tolist()
    values = []
    
    for gate in estimated_gates:
        value = sum([(gate.hs[0][i] - expected[i]) ** 2 for i in range(size)])
        value = np.sqrt(value)
        values.append(value)
    
    fig = physicality_violation_check.make_prob_dist_histogram(
            values, bin_size=bin_size, num_data=num_data, annotation_vlines=[expected[i]]
        )
    title = f"N={num_data[num_data_index]}"
    fig.update_layout(title=title)
    fig.update_xaxes(range=[0, fig.layout.xaxis.range[1]])
    
    return fig

In [13]:
figs = physicality_violation_check.make_graphs_trace_error(estimation_results_list[0], num_data_index=0)

In [17]:
figs[3]

In [73]:
fig = physicality_violation_check.make_graphs_trace_error_sum(estimation_results_list[0], num_data_index=0)

In [76]:
estimation_results = estimation_results_list[0]
num_data_index = 0

estimated_gates = physicality_violation_check._convert_result_to_qoperation(
        estimation_results, num_data_index=num_data_index
    )

In [77]:
physicality_violation_check.get_sorted_eigenvalues_list(estimated_gates)

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'Gate' object has no attribute 'calc_eigenvalues'

In [78]:
gate = estimated_gates[0]

In [156]:
def get_sorted_eigenvalue_gate(gates: List[Gate]) -> list:
        sorted_eigenvalues_list = []
        for gate in gates:
            choi_matrix = gate.to_choi_matrix()
            eigenvals, _ = np.linalg.eig(choi_matrix)
            sorted_eigenvalues = [eig.real for eig in sorted(eigenvals, reverse=True)]
            sorted_eigenvalues_list.append(sorted_eigenvalues)
        return sorted_eigenvalues_list

In [157]:
def get_sorted_eigenvalue_gate(gates: List[Gate]) -> list:
        sorted_eigenvalues_list = []
        for gate in gates:
            choi_matrix = gate.to_choi_matrix()
            eigenvals, _ = np.linalg.eig(choi_matrix)
            sorted_eigenvalues = [eig.real for eig in sorted(eigenvals, reverse=True)]
            sorted_eigenvalues_list.append(sorted_eigenvalues)
        return sorted_eigenvalues_list

def _make_graphs_eigenvalues_gate(estimated_gates: List[Gate],
    num_data: int,
    bin_size: float = 0.0001):
    sorted_eigenvalues_list = get_sorted_eigenvalues_list(estimated_gates)
    sorted_eigenvalues_list = np.array(sorted_eigenvalues_list).T

    dim = estimated_gates[0].dim

    figs = []
    for i, values in enumerate(sorted_eigenvalues_list):
        min_value = min(values)
        max_value = max(values)
        vlines = []
        if (max_value <= 0) or (min_value <= 0 <= max_value) or (abs(min_value) <= abs(max_value - dim)):
            vlines.append(0)
        if (abs(min_value) >= abs(max_value - dim)) or (min_value <= dim <= max_value) or (dim <= min_value):
            vlines.append(dim)

        fig = physicality_violation_check.make_prob_dist_histogram(
            values, bin_size=bin_size, num_data=num_data, annotation_vlines=vlines
        )
        title = f"N={num_data}, i={i}"
        fig.update_layout(title=title)
        figs.append(fig)
        
    return figs

In [158]:
estimation_results = estimation_results_list[0]
num_data_index = 0

estimated_gates = physicality_violation_check._convert_result_to_qoperation(
        estimation_results, num_data_index=num_data_index
    )
figs = make_graphs_eigenvalues_gate(estimated_gates, num_data=100)

In [163]:
figs = physicality_violation_check.make_graphs_eigenvalues(estimation_results, num_data_index=0, true_object=true_object)

In [174]:
from typing import Tuple
from quara.settings import Settings

def get_sum_of_eigenvalues_violation(
    sorted_eigenvalues_list: List[List[float]],
     expected_values=(0, 1), 
) -> Tuple[List[float], List[float]]:
    expected_values = sorted(expected_values)
    if len(expected_values) > 2:
        # TODO: error message
        raise ValueError()
    
    sum_eig_less_list = []
    sum_eig_greater_list = []
    
    eps = Settings.get_atol()
    for i, values in enumerate(sorted_eigenvalues_list):
        eig_less_list = [v for v in values if v < expected_values[0] - eps]
        if eig_less_list:
            sum_eig_less_list.append(np.sum(eig_less_list))

        eig_greater_list = [v for v in values if v >  expected_values[1] + eps]
        if eig_greater_list:
            sum_eig_greater_list.append(np.sum(eig_greater_list))

    return sum_eig_less_list, sum_eig_greater_list

In [175]:
sorted_eigenvalues_list = physicality_violation_check.get_sorted_eigenvalue_gate(estimated_gates)
sum_eig_less_list, sum_eig_greater_list = get_sum_of_eigenvalues_violation(sorted_eigenvalues_list, expected_values=(0, dim))


In [179]:
from typing import Union

In [198]:
def _make_graphs_sum_unphysical_eigenvalues(
    estimated_qobjects: List[Union[State, Gate]], num_data: int, bin_size: float = 0.0001,  expected_values=(0, 1)
) -> List["Figure"]:
    expected_values = list(sorted(expected_values))
    sorted_eigenvalues_list = physicality_violation_check.get_sorted_eigenvalues_list(estimated_gates)
    less_list, greater_list = get_sum_of_eigenvalues_violation(
        sorted_eigenvalues_list, expected_values=expected_values
    )

    n_rep = len(sorted_eigenvalues_list)
    figs = []
    # Figure 1
    xaxis_title_text = f"Sum of unphysical eigenvalues (<{expected_values[0]})"
    n_unphysical = len(less_list)

    fig = physicality_violation_check.make_prob_dist_histogram(
        less_list,
        bin_size=bin_size,
        num_data=num_data,
        annotation_vlines=[expected_values[0]],
        xaxis_title_text=xaxis_title_text,
        title=f"N={num_data}, Nrep={n_rep}",
        additional_title_text=f"<br>Number of unphysical estimates={n_unphysical}",
    )
    print(f"len(less_list) = {len(less_list)}")
    print(f"len(greater_list) = {len(greater_list)}")

    figs.append(fig)

    # Figure 2
    xaxis_title_text = f"Sum of unphysical eigenvalues (>{expected_values[1]})"
    n_unphysical = len(less_list)
    fig = physicality_violation_check.make_prob_dist_histogram(
        greater_list,
        bin_size=bin_size,
        num_data=num_data,
        annotation_vlines=[expected_values[1]],
        xaxis_title_text=xaxis_title_text,
        title=f"N={num_data}, Nrep={n_rep}",
        additional_title_text=f"<br>Number of unphysical estimates={n_unphysical}",
    )

    figs.append(fig)

    return figs

In [202]:
figs = _make_graphs_sum_unphysical_eigenvalues(estimated_gates, num_data=100, expected_values=(0, dim))

n_rep=10
len(less_list) = 10
len(greater_list) = 6
n_rep=6


In [262]:
estimation_results = estimation_results_list[3]
estimated_qobjects = physicality_violation_check._convert_result_to_qoperation(
        estimation_results, num_data_index=1
    )
sorted_eigenvalues_list = physicality_violation_check.get_sorted_eigenvalues_list(estimated_qobjects)
less_list, greater_list = physicality_violation_check.get_sum_of_eigenvalues_violation(
        sorted_eigenvalues_list, expected_values=(0, 2)
    )

In [270]:
sum_eig_less_list = []
sum_eig_greater_list = []
expected_values = (0, 2)

eps = Settings.get_atol()
for i, values in enumerate(sorted_eigenvalues_list):
    eig_less_list = [v for v in values if v < expected_values[0] - eps]
    if eig_less_list:
        print(f"{i}: {eig_less_list=}")
        sum_eig_less_list.append(np.sum(eig_less_list))

    eig_greater_list = [v for v in values if v > expected_values[1] + eps]
    # print(eig_greater_list)
    if eig_greater_list:
        sum_eig_greater_list.append(np.sum(eig_greater_list))

8: eig_less_list=[-1.1611907279241869e-13]
10: eig_less_list=[-1.2983456182592547e-13, -1.884131064254702e-13]
21: eig_less_list=[-1.4138685688188596e-13]
25: eig_less_list=[-1.2863909422404853e-13]
83: eig_less_list=[-1.0731494951837895e-13]
87: eig_less_list=[-1.22897201484707e-13, -1.4529599220017426e-13]
92: eig_less_list=[-1.0345514656654012e-13]


In [278]:
sorted_eigenvalues_list[25]

[1.9669189589670433,
 0.0330810410331723,
 -8.769165401125674e-14,
 -1.2863909422404853e-13]

In [281]:
vals, _ = np.linalg.eig(estimated_qobjects[21].to_choi_matrix())
vals

array([ 1.97468072e+00-5.32989431e-18j,  2.53192793e-02+1.58095153e-17j,
       -1.41386857e-13-1.73408173e-17j, -9.40773071e-14+4.68239184e-18j])

In [263]:
figs = physicality_violation_check.make_graphs_sum_unphysical_eigenvalues(
    estimation_results_list[3], num_data_index=1
)

In [264]:
figs[1]